In [1]:
import pandas as pd
import numpy as np
import torch
from sklearn import preprocessing
from dgl.data import DGLDataset
import dgl
import time
import networkx as nx
import category_encoders as ce
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import torch
import tqdm
import math

from typing import *
from sklearn.preprocessing import StandardScaler, Normalizer
import socket
import struct
import random
from sklearn.model_selection import train_test_split

/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
file_name = "NF-UNSW-NB15-v2.parquet"
data = pd.read_parquet(file_name)

In [3]:
# ADDED
file_name_test = "cleaned_output.csv"
test_data = pd.read_csv(file_name_test)
##

In [4]:
data.Label.value_counts()

Label
0    2295222
1      95053
Name: count, dtype: int64

In [5]:
# ADDED
test_data.Label.value_counts()
##

Label
1    7397
0    4585
Name: count, dtype: int64

In [6]:
data.rename(columns=lambda x: x.strip(), inplace=True)
data['IPV4_SRC_ADDR'] = data["IPV4_SRC_ADDR"].apply(str)
data['L4_SRC_PORT'] = data["L4_SRC_PORT"].apply(str)
data['IPV4_DST_ADDR'] = data["IPV4_DST_ADDR"].apply(str)
data['L4_DST_PORT'] = data["L4_DST_PORT"].apply(str)

In [7]:
# ADDED
test_data.rename(columns=lambda x: x.strip(), inplace=True)
test_data['IPV4_SRC_ADDR'] = test_data["IPV4_SRC_ADDR"].apply(str)
test_data['L4_SRC_PORT'] = test_data["L4_SRC_PORT"].apply(str)
test_data['IPV4_DST_ADDR'] = test_data["IPV4_DST_ADDR"].apply(str)
test_data['L4_DST_PORT'] = test_data["L4_DST_PORT"].apply(str)
##

In [8]:
data.drop(columns=["L4_SRC_PORT", "L4_DST_PORT"], inplace=True)

In [9]:
##ADDED
test_data.drop(columns=["L4_SRC_PORT", "L4_DST_PORT"], inplace=True)
##

In [10]:
data.Attack.unique()

array(['Benign', 'Exploits', 'Generic', 'Fuzzers', 'Backdoor', 'DoS',
       'Reconnaissance', 'Shellcode', 'Worms', 'Analysis'], dtype=object)

In [11]:
##ADDED
test_data.Attack.unique()
##

array(['DoS', 'Benign', 'Fuzzers'], dtype=object)

In [12]:
data = data.groupby(by='Attack').sample(frac=0.1, random_state=13)

In [13]:
# ADDED
# test_data = test_data.groupby(by='Attack').sample(frac=0.1, random_state=13)
##

In [14]:
data.groupby(by="Attack").count()

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Label
Attack,,,,,,,,,,,,,,,,,,,,,
Analysis,230,230,230,230,230,230,230,230,230,230,...,230,230,230,230,230,230,230,230,230,230
Backdoor,217,217,217,217,217,217,217,217,217,217,...,217,217,217,217,217,217,217,217,217,217
Benign,229522,229522,229522,229522,229522,229522,229522,229522,229522,229522,...,229522,229522,229522,229522,229522,229522,229522,229522,229522,229522
DoS,579,579,579,579,579,579,579,579,579,579,...,579,579,579,579,579,579,579,579,579,579
Exploits,3155,3155,3155,3155,3155,3155,3155,3155,3155,3155,...,3155,3155,3155,3155,3155,3155,3155,3155,3155,3155
Fuzzers,2231,2231,2231,2231,2231,2231,2231,2231,2231,2231,...,2231,2231,2231,2231,2231,2231,2231,2231,2231,2231
Generic,1656,1656,1656,1656,1656,1656,1656,1656,1656,1656,...,1656,1656,1656,1656,1656,1656,1656,1656,1656,1656
Reconnaissance,1278,1278,1278,1278,1278,1278,1278,1278,1278,1278,...,1278,1278,1278,1278,1278,1278,1278,1278,1278,1278
Shellcode,143,143,143,143,143,143,143,143,143,143,...,143,143,143,143,143,143,143,143,143,143


In [15]:
# ADDED
test_data.groupby(by="Attack").count()
##

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Label
Attack,,,,,,,,,,,,,,,,,,,,,
Benign,4585,4585,4585,4585,4585,4585,4585,4585,4585,4585,...,4585,4585,4585,4585,4585,4585,4585,4585,4585,4585
DoS,4046,4046,4046,4046,4046,4046,4046,4046,4046,4046,...,4046,4046,4046,4046,4046,4046,4046,4046,4046,4046
Fuzzers,3351,3351,3351,3351,3351,3351,3351,3351,3351,3351,...,3351,3351,3351,3351,3351,3351,3351,3351,3351,3351


In [73]:
X = data.drop(columns=["Attack", "Label"])
y = data[["Attack", "Label"]]

X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=13, stratify=y)
# ADDED
X_test = test_data.drop(columns=["Attack", "Label"])
y_test = test_data[["Attack", "Label"]]

X_train.shape, X_test.shape, y_train.shape, y_test.shape
##

((167318, 41), (11982, 41), (167318, 2), (11982, 2))

In [17]:
encoder = ce.TargetEncoder(cols=['TCP_FLAGS','L7_PROTO','PROTOCOL',
                                  'CLIENT_TCP_FLAGS','SERVER_TCP_FLAGS','ICMP_TYPE',
                                  'ICMP_IPV4_TYPE','DNS_QUERY_ID','DNS_QUERY_TYPE',
                                  'FTP_COMMAND_RET_CODE'])
encoder.fit(X_train, y_train.Label)

# Transform on training set
X_train = encoder.transform(X_train)

# Transform on testing set
X_test = encoder.transform(X_test)

In [18]:
X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test.replace([np.inf, -np.inf], np.nan, inplace=True)
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

In [19]:
scaler = Normalizer()
cols_to_norm = list(set(list(X_train.iloc[:, 2:].columns))) # Ignore first two as the represents IP addresses
scaler.fit(X_train[cols_to_norm])

# Transform on training set
X_train[cols_to_norm] = scaler.transform(X_train[cols_to_norm])
X_train['h'] = X_train.iloc[:, 2:].values.tolist()

# Transform on testing set
X_test[cols_to_norm] = scaler.transform(X_test[cols_to_norm])
X_test['h'] = X_test.iloc[:, 2:].values.tolist()

train = pd.concat([X_train, y_train], axis=1)
test = pd.concat([X_test, y_test], axis=1)

In [20]:
X_train.head()

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,h
220614,59.166.0.4,149.171.126.1,4.802078e-09,3.048648e-09,3.994566e-05,7.652425e-07,0.000072,1.071339e-06,1.980229e-09,1.980360e-09,...,0.000000e+00,1.329685e-03,1.329685e-03,4.043553e-10,4.043553e-10,6.314168e-09,6.314212e-09,0.000000,3.618463e-09,"[4.802078220820267e-09, 3.0486475517277593e-09..."
329844,59.166.0.1,149.171.126.8,9.025364e-09,8.052833e-09,1.079121e-04,8.300931e-07,0.000063,8.300931e-07,1.397062e-08,1.396955e-08,...,0.000000e+00,0.000000e+00,0.000000e+00,1.181795e-08,1.182599e-08,8.561578e-09,8.561639e-09,0.000000,9.271523e-09,"[9.025363504432674e-09, 8.052833119952719e-09,..."
1093665,59.166.0.3,149.171.126.8,3.068685e-13,8.437668e-13,1.638983e-07,3.061234e-09,0.000010,7.129838e-09,1.188386e-13,1.274923e-13,...,6.875550e-09,6.372843e-07,7.080937e-08,4.545956e-14,5.191931e-14,4.034960e-13,4.034988e-13,0.000000,4.369547e-13,"[3.0686849809527964e-13, 8.437667985448175e-13..."
1764268,59.166.0.4,149.171.126.0,2.361402e-08,2.106949e-08,7.927293e-05,1.085931e-06,0.000097,1.085931e-06,3.655283e-08,3.655002e-08,...,0.000000e+00,0.000000e+00,0.000000e+00,3.092057e-08,3.094159e-08,1.878071e-08,3.577436e-10,0.000033,2.425808e-08,"[2.3614022984160315e-08, 2.1069487814953574e-0..."
1525176,59.166.0.9,149.171.126.5,2.309362e-10,2.856102e-10,1.717880e-05,2.649686e-07,0.000124,2.796890e-07,8.943290e-11,9.594530e-11,...,7.360238e-08,1.598644e-04,1.065762e-04,5.174884e-12,5.174884e-12,3.036539e-10,3.036561e-10,0.000000,3.288336e-10,"[2.309362039695701e-10, 2.856102367918858e-10,..."


In [21]:
lab_enc = preprocessing.LabelEncoder()
lab_enc.fit(data["Attack"])

# Transform on training set
train["Attack"] = lab_enc.transform(train["Attack"])

# Transform on testing set
test["Attack"] = lab_enc.transform(test["Attack"])

In [22]:
# Training graph

train_g = nx.from_pandas_edgelist(train, "IPV4_SRC_ADDR", "IPV4_DST_ADDR",
           ["h", "Label", "Attack"], create_using=nx.MultiGraph())
print(train_g) # 39 nodes, 167318 edges
train_g = train_g.to_directed()
print(train_g) # 39 nodes, 334636 edges (doubled because of directed graph)
train_g = dgl.from_networkx(train_g, edge_attrs=['h', 'Attack', 'Label'])
nfeat_weight = torch.ones([train_g.number_of_nodes(),
train_g.edata['h'].shape[1]])
train_g.ndata['h'] = nfeat_weight

# Testing graph
test_g = nx.from_pandas_edgelist(test, "IPV4_SRC_ADDR", "IPV4_DST_ADDR",
            ["h", "Label", "Attack"], create_using=nx.MultiGraph())

test_g = test_g.to_directed()
test_g = dgl.from_networkx(test_g, edge_attrs=['h', 'Attack', 'Label'])
nfeat_weight = torch.ones([test_g.number_of_nodes(),
test_g.edata['h'].shape[1]])
test_g.ndata['h'] = nfeat_weight

MultiGraph with 39 nodes and 167318 edges
MultiDiGraph with 39 nodes and 334636 edges


In [23]:
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import tqdm
import gc

class SAGELayer(nn.Module):
    def __init__(self, ndim_in, edims, ndim_out, activation):
      super(SAGELayer, self).__init__()
      self.W_apply = nn.Linear(ndim_in + edims , ndim_out)
      self.activation = F.relu
      self.W_edge = nn.Linear(128 * 2, 256)
      self.reset_parameters()

    def reset_parameters(self):
      gain = nn.init.calculate_gain('relu')
      nn.init.xavier_uniform_(self.W_apply.weight, gain=gain)

    def message_func(self, edges):
      return {'m':  edges.data['h']}

    def forward(self, g_dgl, nfeats, efeats):
      with g_dgl.local_scope():
        g = g_dgl
        g.ndata['h'] = nfeats
        g.edata['h'] = efeats
        g.update_all(self.message_func, fn.mean('m', 'h_neigh'))
        g.ndata['h'] = F.relu(self.W_apply(torch.cat([g.ndata['h'], g.ndata['h_neigh']], 2)))

        # Compute edge embeddings
        u, v = g.edges()
        edge = self.W_edge(torch.cat((g.srcdata['h'][u], g.dstdata['h'][v]), 2))
        return g.ndata['h'], edge

In [24]:
class SAGE(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim,  activation):
      super(SAGE, self).__init__()
      self.layers = nn.ModuleList()
      self.layers.append(SAGELayer(ndim_in, edim, 128, F.relu))

    def forward(self, g, nfeats, efeats, corrupt=False):
      if corrupt:
        e_perm = torch.randperm(g.number_of_edges())
        #n_perm = torch.randperm(g.number_of_nodes())
        efeats = efeats[e_perm]
        #nfeats = nfeats[n_perm]
      for i, layer in enumerate(self.layers):
        #nfeats = layer(g, nfeats, efeats)
        nfeats, e_feats = layer(g, nfeats, efeats)
      #return nfeats.sum(1)
      return nfeats.sum(1), e_feats.sum(1)

In [25]:
class Discriminator(nn.Module):
    def __init__(self, n_hidden):
      super(Discriminator, self).__init__()
      self.weight = nn.Parameter(torch.Tensor(n_hidden, n_hidden))
      self.reset_parameters()

    def uniform(self, size, tensor):
      bound = 1.0 / math.sqrt(size)
      if tensor is not None:
        tensor.data.uniform_(-bound, bound)

    def reset_parameters(self):
      size = self.weight.size(0)
      self.uniform(size, self.weight)

    def forward(self, features, summary):
      features = torch.matmul(features, torch.matmul(self.weight, summary))
      return features

In [26]:
class DGI(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim, activation):
      super(DGI, self).__init__()
      self.encoder = SAGE(ndim_in, ndim_out, edim,  F.relu)
      #self.discriminator = Discriminator(128)
      self.discriminator = Discriminator(256)
      self.loss = nn.BCEWithLogitsLoss()

    def forward(self, g, n_features, e_features):
      positive = self.encoder(g, n_features, e_features, corrupt=False)
      negative = self.encoder(g, n_features, e_features, corrupt=True)
      self.loss = nn.BCEWithLogitsLoss()

    def forward(self, g, n_features, e_features):
      positive = self.encoder(g, n_features, e_features, corrupt=False)
      negative = self.encoder(g, n_features, e_features, corrupt=True)

      positive = positive[1]
      negative = negative[1]

      summary = torch.sigmoid(positive.mean(dim=0))

      positive = self.discriminator(positive, summary)
      negative = self.discriminator(negative, summary)

      l1 = self.loss(positive, torch.ones_like(positive))
      l2 = self.loss(negative, torch.zeros_like(negative))

      return l1 + l2

In [27]:
ndim_in = train_g.ndata['h'].shape[1]
hidden_features = 128
ndim_out = 128
num_layers = 1
edim = train_g.edata['h'].shape[1]
learning_rate = 1e-3
epochs = 4000

In [28]:
dgi = DGI(ndim_in,
    ndim_out,
    edim,
    F.relu)

dgi = dgi.to('cuda')

dgi_optimizer = torch.optim.Adam(dgi.parameters(),
                lr=1e-3,
                weight_decay=0.)

In [29]:
# Format node and edge features for E-GraphSAGE
train_g.ndata['h'] = torch.reshape(train_g.ndata['h'],
                                   (train_g.ndata['h'].shape[0], 1,
                                    train_g.ndata['h'].shape[1]))

train_g.edata['h'] = torch.reshape(train_g.edata['h'],
                                   (train_g.edata['h'].shape[0], 1,
                                    train_g.edata['h'].shape[1]))

In [30]:
# Convert to GPU
train_g = train_g.to('cuda')

In [31]:
# cnt_wait = 0
# best = 1e9
# best_t = 0
# dur = []
# node_features = train_g.ndata['h'] 
# edge_features = train_g.edata['h']

# for epoch in range(epochs):
#     dgi.train()
#     if epoch >= 3:
#         t0 = time.time()

#     dgi_optimizer.zero_grad()
#     loss = dgi(train_g, node_features, edge_features)
#     loss.backward()
#     dgi_optimizer.step()

#     if loss < best:
#         best = loss
#         best_t = epoch
#         cnt_wait = 0
#         torch.save(dgi.state_dict(), 'best_dgi_UNSW.pkl')
#     else:
#         cnt_wait += 1

#   # if cnt_wait == patience:
#   #     print('Early stopping!')
#   #     break

#     if epoch >= 3:
#         dur.append(time.time() - t0)

#     if epoch % 50 == 0:

#         print("Epoch {:05d} | Time(s) {:.4f} | Loss {:.4f} | "
#             "ETputs(KTEPS) {:.2f}".format(epoch, np.mean(dur),
#               loss.item(),
#               train_g.num_edges() / np.mean(dur) / 1000))

In [32]:
dgi.load_state_dict(torch.load('best_dgi_UNSW.pkl'))

<All keys matched successfully>

In [33]:
training_emb = dgi.encoder(train_g, train_g.ndata['h'], train_g.edata['h'])[1]
training_emb = training_emb.detach().cpu().numpy()

In [34]:
test_g.ndata['h'] = torch.reshape(test_g.ndata['h'],
                                   (test_g.ndata['h'].shape[0], 1,
                                    test_g.ndata['h'].shape[1]))



test_g.edata['h'] = torch.reshape(test_g.edata['h'],
                                   (test_g.edata['h'].shape[0], 1,
                                    test_g.edata['h'].shape[1]))

In [35]:
# Convert to GPU
test_g = test_g.to('cuda')

In [36]:
testing_emb = dgi.encoder(test_g, test_g.ndata['h'], test_g.edata['h'])[1]
testing_emb = testing_emb.detach().cpu().numpy()

In [37]:
df_train = pd.DataFrame(training_emb, )
df_train["Attack"] = lab_enc.inverse_transform(
        train_g.edata['Attack'].detach().cpu().numpy())
df_train["Label"] = train_g.edata['Label'].detach().cpu().numpy()

df_test = pd.DataFrame(testing_emb, )
df_test["Attack"] = lab_enc.inverse_transform(
        test_g.edata['Attack'].detach().cpu().numpy())
df_test["Label"] = test_g.edata['Label'].detach().cpu().numpy()

In [38]:
df_train

,0,1,2,3,4,5,6,7,8,9,...,248,249,250,251,252,253,254,255,Attack,Label
0,-0.042790,0.055066,-0.016933,0.062138,-0.022476,0.043912,-0.032069,0.024231,-0.004287,0.053993,...,0.010456,0.011092,-0.011088,0.007600,0.072127,0.058860,-0.033013,0.023536,Benign,0
1,-0.042790,0.055066,-0.016933,0.062138,-0.022476,0.043912,-0.032069,0.024231,-0.004287,0.053993,...,0.010456,0.011092,-0.011088,0.007600,0.072127,0.058860,-0.033013,0.023536,Benign,0
2,-0.042790,0.055066,-0.016933,0.062138,-0.022476,0.043912,-0.032069,0.024231,-0.004287,0.053993,...,0.010456,0.011092,-0.011088,0.007600,0.072127,0.058860,-0.033013,0.023536,Benign,0
3,-0.042790,0.055066,-0.016933,0.062138,-0.022476,0.043912,-0.032069,0.024231,-0.004287,0.053993,...,0.010456,0.011092,-0.011088,0.007600,0.072127,0.058860,-0.033013,0.023536,Benign,0
4,-0.042790,0.055066,-0.016933,0.062138,-0.022476,0.043912,-0.032069,0.024231,-0.004287,0.053993,...,0.010456,0.011092,-0.011088,0.007600,0.072127,0.058860,-0.033013,0.023536,Benign,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334631,-0.025799,0.075020,-0.028325,0.068261,-0.012427,0.056157,-0.051078,0.006709,0.006530,0.063153,...,0.022400,0.004744,-0.025576,0.044644,0.065002,0.055548,-0.056300,0.046742,Benign,0
334632,-0.025799,0.075020,-0.028325,0.068261,-0.012427,0.056157,-0.051078,0.006709,0.006530,0.063153,...,0.022400,0.004744,-0.025576,0.044644,0.065002,0.055548,-0.056300,0.046742,Benign,0
334633,-0.025799,0.075020,-0.028325,0.068261,-0.012427,0.056157,-0.051078,0.006709,0.006530,0.063153,...,0.022400,0.004744,-0.025576,0.044644,0.065002,0.055548,-0.056300,0.046742,Benign,0
334634,-0.025799,0.075020,-0.028325,0.068261,-0.012427,0.056157,-0.051078,0.006709,0.006530,0.063153,...,0.022400,0.004744,-0.025576,0.044644,0.065002,0.055548,-0.056300,0.046742,Benign,0


# Embeddings CBLOF  Embeddings

In [39]:
import torch
import dgl
import numpy as np
import pandas as pd
import torch.optim as optim
import time
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn.ensemble import IsolationForest
import gc

from tqdm import tqdm
import itertools

In [40]:
benign_train_samples = df_train[df_train.Label == 0].drop(columns=["Label", "Attack"])
normal_train_samples = df_train.drop(columns=["Label", "Attack"])

train_labels = df_train["Label"]
test_labels = df_test["Label"]

test_samples = df_test.drop(columns=["Label", "Attack"])

In [41]:
test_samples

,0,1,2,3,4,5,6,7,8,9,...,246,247,248,249,250,251,252,253,254,255
0,-0.056488,0.043071,0.002685,0.059874,-0.003218,0.027074,-0.022901,0.032087,-0.007770,0.047968,...,-0.057221,0.019634,-0.008637,-0.013681,-0.005194,-0.013679,0.048709,0.048467,-0.052041,0.028930
1,-0.056488,0.043071,0.002685,0.059874,-0.003218,0.027074,-0.022901,0.032087,-0.007770,0.047968,...,-0.057221,0.019634,-0.008637,-0.013681,-0.005194,-0.013679,0.048709,0.048467,-0.052041,0.028930
2,-0.059438,0.041020,0.002833,0.059534,0.000370,0.033774,-0.030775,0.028989,-0.000397,0.046140,...,-0.051450,0.024531,-0.006667,-0.010680,-0.009451,-0.001782,0.055676,0.051359,-0.052547,0.031350
3,-0.059438,0.041020,0.002833,0.059534,0.000370,0.033774,-0.030775,0.028989,-0.000397,0.046140,...,-0.051450,0.024531,-0.006667,-0.010680,-0.009451,-0.001782,0.055676,0.051359,-0.052547,0.031350
4,-0.059438,0.041020,0.002833,0.059534,0.000370,0.033774,-0.030775,0.028989,-0.000397,0.046140,...,-0.051450,0.024531,-0.006667,-0.010680,-0.009451,-0.001782,0.055676,0.051359,-0.052547,0.031350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23959,-0.059438,0.041021,0.002833,0.059536,0.000369,0.033772,-0.030773,0.028988,-0.000399,0.046139,...,-0.051451,0.024530,-0.006667,-0.010681,-0.009452,-0.001783,0.055673,0.051360,-0.052546,0.031350
23960,-0.059438,0.041021,0.002833,0.059536,0.000369,0.033772,-0.030773,0.028988,-0.000399,0.046139,...,-0.051451,0.024530,-0.006667,-0.010681,-0.009452,-0.001783,0.055673,0.051360,-0.052546,0.031350
23961,-0.059438,0.041021,0.002833,0.059536,0.000369,0.033772,-0.030773,0.028988,-0.000399,0.046139,...,-0.051451,0.024530,-0.006667,-0.010681,-0.009452,-0.001783,0.055673,0.051360,-0.052546,0.031350
23962,0.013232,0.125959,-0.095555,0.081425,-0.079256,0.092037,-0.049377,-0.007579,0.008550,0.094855,...,-0.038782,0.021645,0.065187,0.074931,-0.022693,0.076644,0.126539,0.094185,0.025648,0.029415


In [42]:
df_raw_train = pd.concat([X_train.drop(columns=["IPV4_SRC_ADDR","IPV4_DST_ADDR", "h"]), y_train], axis=1)
df_raw_test = pd.concat([X_test.drop(columns=["IPV4_SRC_ADDR","IPV4_DST_ADDR", "h"]), y_test], axis=1)

In [43]:
df_raw_train

,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,...,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Attack,Label
220614,4.802078e-09,3.048648e-09,3.994566e-05,7.652425e-07,0.000072,1.071339e-06,1.980229e-09,1.980360e-09,0.000000e+00,0.000000,...,1.329685e-03,1.329685e-03,4.043553e-10,4.043553e-10,6.314168e-09,6.314212e-09,0.000000,3.618463e-09,Benign,0
329844,9.025364e-09,8.052833e-09,1.079121e-04,8.300931e-07,0.000063,8.300931e-07,1.397062e-08,1.396955e-08,1.452568e-08,0.000000,...,0.000000e+00,0.000000e+00,1.181795e-08,1.182599e-08,8.561578e-09,8.561639e-09,0.000000,9.271523e-09,Benign,0
1093665,3.068685e-13,8.437668e-13,1.638983e-07,3.061234e-09,0.000010,7.129838e-09,1.188386e-13,1.274923e-13,0.000000e+00,0.000042,...,6.372843e-07,7.080937e-08,4.545956e-14,5.191931e-14,4.034960e-13,4.034988e-13,0.000000,4.369547e-13,Benign,0
1764268,2.361402e-08,2.106949e-08,7.927293e-05,1.085931e-06,0.000097,1.085931e-06,3.655283e-08,3.655002e-08,3.800509e-08,0.000000,...,0.000000e+00,0.000000e+00,3.092057e-08,3.094159e-08,1.878071e-08,3.577436e-10,0.000033,2.425808e-08,Benign,0
1525176,2.309362e-10,2.856102e-10,1.717880e-05,2.649686e-07,0.000124,2.796890e-07,8.943290e-11,9.594530e-11,0.000000e+00,0.000000,...,1.598644e-04,1.065762e-04,5.174884e-12,5.174884e-12,3.036539e-10,3.036561e-10,0.000000,3.288336e-10,Benign,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1525437,1.940855e-09,1.232171e-09,1.123952e-04,2.041300e-06,0.000155,2.165015e-06,8.003488e-10,8.004016e-10,0.000000e+00,0.000000,...,1.253977e-03,1.074837e-03,1.006141e-10,1.006141e-10,2.551995e-09,2.552013e-09,0.000000,0.000000e+00,Benign,0
1636406,2.617265e-08,2.335241e-08,7.823357e-05,1.203593e-06,0.000097,1.203593e-06,4.051340e-08,4.051029e-08,4.212302e-08,0.000000,...,0.000000e+00,0.000000e+00,3.427088e-08,3.429417e-08,2.053465e-08,3.965058e-10,0.000036,2.688649e-08,Benign,0
152133,4.439600e-10,5.490674e-10,5.999424e-06,1.131967e-07,0.000125,1.697950e-07,1.719290e-10,3.618107e-09,0.000000e+00,0.000000,...,1.434202e-04,8.195440e-05,8.057850e-10,8.063327e-10,5.837552e-10,5.837593e-10,0.000000,6.321614e-10,Benign,0
1164696,2.451807e-09,1.556554e-09,9.775598e-05,1.797272e-06,0.000137,1.953557e-06,1.011050e-09,1.011116e-09,0.000000e+00,0.000000,...,1.244650e-03,1.131500e-03,4.425931e-10,4.423147e-10,3.223837e-09,3.223860e-09,0.000000,0.000000e+00,Benign,0


In [70]:
df_raw_test

,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,...,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Attack,Label
0,1.204563e-08,7.743290e-08,0.000086,2.769692e-07,0.000091,2.769692e-07,1.864578e-08,1.864434e-08,1.938658e-08,0.000000,...,0.000000,0.000000,1.577274e-08,1.578346e-08,1.142664e-08,1.142672e-08,0.0,1.237416e-08,DoS,1
1,4.279156e-08,1.176598e-07,0.000109,2.727645e-06,0.000060,1.363823e-06,1.161399e-08,5.422915e-08,6.909234e-08,0.000000,...,0.000000,0.087612,7.766644e-08,7.771923e-08,5.626586e-08,5.626626e-08,0.0,6.093155e-08,DoS,1
2,8.461442e-08,2.326561e-07,0.000216,5.393544e-06,0.000119,2.696772e-06,2.296506e-08,1.072307e-07,1.366206e-07,0.000003,...,0.000000,0.173241,1.535747e-07,1.536791e-07,1.112580e-07,1.112588e-07,0.0,1.204838e-07,DoS,1
3,8.461442e-08,2.326561e-07,0.000216,5.393544e-06,0.000119,2.696772e-06,2.296506e-08,1.072307e-07,1.366206e-07,0.000003,...,0.000000,0.173241,1.535747e-07,1.536791e-07,1.112580e-07,1.112588e-07,0.0,1.204838e-07,DoS,1
4,8.461442e-08,2.326561e-07,0.000216,5.393544e-06,0.000119,2.696772e-06,2.296506e-08,1.072307e-07,1.366206e-07,0.000003,...,0.000000,0.173241,1.535747e-07,1.536791e-07,1.112580e-07,1.112588e-07,0.0,1.204838e-07,DoS,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11977,6.745802e-09,8.342867e-09,0.000112,2.149975e-06,0.000055,1.289985e-06,1.834458e-07,5.497574e-08,1.027710e-07,0.000000,...,0.003522,0.003522,1.154021e-07,1.161291e-07,8.869935e-09,8.869998e-09,0.0,9.605450e-09,Fuzzers,1
11978,6.745802e-09,8.342867e-09,0.000112,2.149975e-06,0.000055,1.289985e-06,1.834458e-07,5.497574e-08,1.027710e-07,0.000000,...,0.003522,0.003522,1.154021e-07,1.161291e-07,8.869935e-09,8.869998e-09,0.0,9.605450e-09,Fuzzers,1
11979,6.745802e-09,8.342867e-09,0.000112,2.149975e-06,0.000055,1.289985e-06,1.834458e-07,5.497574e-08,1.027710e-07,0.000000,...,0.003522,0.003522,1.154021e-07,1.161291e-07,8.869935e-09,8.869998e-09,0.0,9.605450e-09,Fuzzers,1
11980,5.832307e-09,3.702699e-09,0.000056,1.301183e-06,0.000081,1.487066e-06,1.214802e-07,1.035455e-07,8.459357e-08,0.000000,...,0.003045,0.003045,1.606367e-07,1.606367e-07,7.668797e-09,7.668851e-09,0.0,8.304710e-09,Fuzzers,1


In [44]:
raw_benign_train_samples = df_raw_train[df_raw_train.Label == 0].drop(columns=["Label", "Attack"])
raw_normal_train_samples = df_raw_train.drop(columns=["Label", "Attack"])

raw_train_labels = df_raw_train["Label"]
raw_test_labels = df_raw_test["Label"]

raw_test_samples = df_raw_test.drop(columns=["Label", "Attack"])

In [45]:
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]

In [75]:
from pyod.models.cblof import CBLOF
n_est = [2,3,5,7,9,10]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = CBLOF(n_clusters=n_est, contamination=con)
    clf_if.fit(benign_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [01:01<00:00,  1.72s/it]

{'n_estimators': 2, 'con': 0.04}
0.38170184220031994
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000      9170
           1     0.6173    1.0000    0.7634     14794

    accuracy                         0.6173     23964
   macro avg     0.3087    0.5000    0.3817     23964
weighted avg     0.3811    0.6173    0.4713     23964




/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
 

In [47]:
n_est = [2,3,5,7,9,10]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = CBLOF(n_clusters=n_est, contamination=con)
    clf_if.fit(normal_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [01:02<00:00,  1.74s/it]

{'n_estimators': 2, 'con': 0.1}
0.38170184220031994
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000      9170
           1     0.6173    1.0000    0.7634     14794

    accuracy                         0.6173     23964
   macro avg     0.3087    0.5000    0.3817     23964
weighted avg     0.3811    0.6173    0.4713     23964




/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
 

In [48]:
###  CBLOF RAW

In [49]:
from pyod.models.cblof import CBLOF

n_est = [2,3,5,7,9,10]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    try:
        clf_b = CBLOF(n_clusters=n_est, contamination=con)
        clf_b.fit(raw_benign_train_samples)
    except ValueError as e:
        print(n_est)
        continue  
   
    y_pred = clf_b.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                        "con": con
                }
        bs = test_pred
    del clf_b
    gc.collect()

  

print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

  6%|▌         | 2/36 [00:00<00:06,  5.23it/s]

2
2


 11%|█         | 4/36 [00:00<00:06,  5.22it/s]

2
2


 17%|█▋        | 6/36 [00:01<00:05,  5.13it/s]

2
2


100%|██████████| 36/36 [00:14<00:00,  2.49it/s]

{'n_estimators': 3, 'con': 0.2}
0.9124258376608707
              precision    recall  f1-score   support

           0     0.8171    1.0000    0.8994      4585
           1     1.0000    0.8613    0.9255      7397

    accuracy                         0.9144     11982
   macro avg     0.9086    0.9306    0.9124     11982
weighted avg     0.9300    0.9144    0.9155     11982



In [50]:
n_est = [2,3,5,7,9,10]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    try:
        clf_if = CBLOF(n_clusters=n_est, contamination=con)
        clf_if.fit(raw_normal_train_samples)
    except ValueError as e:
        print(n_est)
        continue  
    
    y_pred = clf_if.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est
                }
        bs = test_pred
    del clf_if
    gc.collect()

        

print("benign only")
print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

  6%|▌         | 2/36 [00:00<00:05,  5.85it/s]

2
2


 11%|█         | 4/36 [00:00<00:05,  5.35it/s]

2
2


 17%|█▋        | 6/36 [00:01<00:05,  5.25it/s]

...

3


 31%|███       | 11/36 [00:02<00:07,  3.29it/s]

3


100%|██████████| 36/36 [00:13<00:00,  2.59it/s]

benign only
{'n_estimators': 3}
0.9118440245531516
              precision    recall  f1-score   support

           0     0.8161    1.0000    0.8988      4585
           1     1.0000    0.8603    0.9249      7397

    accuracy                         0.9138     11982
   macro avg     0.9081    0.9302    0.9118     11982
weighted avg     0.9296    0.9138    0.9149     11982



In [51]:
# HBOS  Embeddings

In [52]:
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]

In [53]:
from pyod.models.hbos import HBOS

n_est = [5,10,15,20,25,30]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = HBOS(n_bins=n_est, contamination=con)
    clf_if.fit(benign_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [01:54<00:00,  3.19s/it]

{'n_estimators': 5, 'con': 0.01}
0.38170184220031994
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000      9170
           1     0.6173    1.0000    0.7634     14794

    accuracy                         0.6173     23964
   macro avg     0.3087    0.5000    0.3817     23964
weighted avg     0.3811    0.6173    0.4713     23964




/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
 

In [54]:
n_est = [5,10,15,20,25,30]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = HBOS(n_bins=n_est, contamination=con)
    clf_if.fit(normal_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [01:57<00:00,  3.27s/it]

{'n_estimators': 5, 'con': 0.001}
0.38170184220031994
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000      9170
           1     0.6173    1.0000    0.7634     14794

    accuracy                         0.6173     23964
   macro avg     0.3087    0.5000    0.3817     23964
weighted avg     0.3811    0.6173    0.4713     23964




/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
 

In [55]:
##  HBOS  RAw

In [56]:
from pyod.models.cblof import CBLOF

n_est = [5,10,15,20,25,30]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    try:
        clf_b = HBOS(n_bins=n_est, contamination=con)
        clf_b.fit(raw_benign_train_samples)
    except ValueError as e:
        print(n_est)
        continue  
   
    y_pred = clf_b.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                        "con": con
                }
        bs = test_pred
    del clf_b
    gc.collect()

  

print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 36/36 [00:11<00:00,  3.14it/s]

{'n_estimators': 30, 'con': 0.2}
0.9430024050657181
              precision    recall  f1-score   support

           0     0.8916    0.9756    0.9317      4585
           1     0.9839    0.9265    0.9543      7397

    accuracy                         0.9453     11982
   macro avg     0.9377    0.9510    0.9430     11982
weighted avg     0.9486    0.9453    0.9457     11982



In [57]:
n_est = [5,10,15,20,25,30]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    try:
        clf_if = HBOS(n_bins=n_est, contamination=con)
        clf_if.fit(raw_normal_train_samples)
    except ValueError as e:
        print(n_est)
        continue  
    
    y_pred = clf_if.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est
                }
        bs = test_pred
    del clf_if
    gc.collect()

        

print("benign only")
print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 36/36 [00:11<00:00,  3.25it/s]

benign only
{'n_estimators': 15}
0.9287364249267158
              precision    recall  f1-score   support

           0     0.8537    0.9889    0.9163      4585
           1     0.9924    0.8950    0.9411      7397

    accuracy                         0.9309     11982
   macro avg     0.9230    0.9419    0.9287     11982
weighted avg     0.9393    0.9309    0.9316     11982



In [58]:
##  PCA  Emb

In [59]:
from pyod.models.pca import PCA
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(benign_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [00:55<00:00,  1.54s/it]

{'n_estimators': 5, 'con': 0.01}
0.38170184220031994
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000      9170
           1     0.6173    1.0000    0.7634     14794

    accuracy                         0.6173     23964
   macro avg     0.3087    0.5000    0.3817     23964
weighted avg     0.3811    0.6173    0.4713     23964




/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
 

In [60]:
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(normal_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [00:57<00:00,  1.59s/it]

{'n_estimators': 5, 'con': 0.001}
0.38170184220031994
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000      9170
           1     0.6173    1.0000    0.7634     14794

    accuracy                         0.6173     23964
   macro avg     0.3087    0.5000    0.3817     23964
weighted avg     0.3811    0.6173    0.4713     23964




/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
 

In [61]:
##  PCA  RAw

In [62]:
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(raw_benign_train_samples)
   
    y_pred = clf_if.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                        "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()

  

print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 36/36 [00:09<00:00,  3.61it/s]

{'n_estimators': 5, 'con': 0.2}
0.9367086364526109
              precision    recall  f1-score   support

           0     0.8700    0.9878    0.9251      4585
           1     0.9917    0.9085    0.9483      7397

    accuracy                         0.9388     11982
   macro avg     0.9308    0.9481    0.9367     11982
weighted avg     0.9451    0.9388    0.9394     11982



In [63]:
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(raw_normal_train_samples)

    y_pred = clf_if.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est
                }
        bs = test_pred
    del clf_if
    gc.collect()

        

print("benign only")
print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 36/36 [00:09<00:00,  3.61it/s]

benign only
{'n_estimators': 5}
0.9273403656329091
              precision    recall  f1-score   support

           0     0.8582    0.9780    0.9142      4585
           1     0.9851    0.8998    0.9405      7397

    accuracy                         0.9297     11982
   macro avg     0.9216    0.9389    0.9273     11982
weighted avg     0.9365    0.9297    0.9304     11982



In [64]:
##  IF  Emb

In [65]:
from sklearn.ensemble import IsolationForest
n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(benign_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 24/24 [00:36<00:00,  1.52s/it]

{'n_estimators': 20, 'con': 0.04}
0.38170184220031994
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000      9170
           1     0.6173    1.0000    0.7634     14794

    accuracy                         0.6173     23964
   macro avg     0.3087    0.5000    0.3817     23964
weighted avg     0.3811    0.6173    0.4713     23964




/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
 

In [66]:
from sklearn.ensemble import IsolationForest

n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(normal_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 24/24 [00:37<00:00,  1.58s/it]

{'n_estimators': 50, 'con': 0.04}
0.382145250922055
              precision    recall  f1-score   support

           0     0.6667    0.0004    0.0009      9170
           1     0.6174    0.9999    0.7634     14794

    accuracy                         0.6174     23964
   macro avg     0.6420    0.5002    0.3821     23964
weighted avg     0.6363    0.6174    0.4716     23964



In [67]:
##  IF  Raw

In [68]:
from sklearn.ensemble import IsolationForest

n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(raw_benign_train_samples.to_numpy())
   
    y_pred = clf_if.predict(raw_test_samples.to_numpy())
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                        "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()

  

print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 24/24 [00:13<00:00,  1.79it/s]

{'n_estimators': 100, 'con': 0.1}
0.9569962548894806
              precision    recall  f1-score   support

           0     0.9867    0.9077    0.9456      4585
           1     0.9455    0.9924    0.9684      7397

    accuracy                         0.9600     11982
   macro avg     0.9661    0.9501    0.9570     11982
weighted avg     0.9613    0.9600    0.9597     11982



In [69]:
n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(raw_normal_train_samples.to_numpy())

    y_pred = clf_if.predict(raw_test_samples.to_numpy())
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est
                }
        bs = test_pred
    del clf_if
    gc.collect()

        

print("benign only")
print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 24/24 [00:13<00:00,  1.77it/s]

benign only
{'n_estimators': 100}
0.9334139613951351
              precision    recall  f1-score   support

           0     0.9609    0.8742    0.9155      4585
           1     0.9261    0.9780    0.9513      7397

    accuracy                         0.9382     11982
   macro avg     0.9435    0.9261    0.9334     11982
weighted avg     0.9394    0.9382    0.9376     11982

